In [1]:
import sys
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
import os
import inspect
import pandas as pd

project_path = os.path.abspath( os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe()))) + "/../")
sys.path.insert(0, project_path)

from Program.Objects.Human import *
from Program.Objects.Issue import *
from Program.Objects.Machine import *
from Program.Objects.MaintenanceWorkOrder import *
from Program.Objects.Tag import *

from Program.Database.Database_Properties import *
from Program.Database.Database import *
from Program.Database.Main import *

from Program.KPI.KPI import *


from Program.Others.MyDate import *

ModuleNotFoundError: No module named 'DatabaseStorage'

# DATABASE

## FILES

### NIST server

### Greystone

In [2]:
PassWord = 'GREYSTONE!!'
#file = os.path.join(project_path , 'Data_CSV', 'GreyStone_Data.csv')
file = os.path.join(project_path , 'Data_CSV', 'GS_tokenized.csv')
localization = {NodeHuman.VALUE_TECHNICIAN.value: 15,
                NodeHuman.VALUE_OPERATOR.value: 13,

                NodeTag.VALUE_ITEM.value: 17,
                NodeTag.VALUE_PROBLEM.value: 19,
                NodeTag.VALUE_SOLUTION.value: 20,
                NodeTag.VALUE_NA.value: 18,
                NodeTag.VALUE_UNKNOWN.value: 21,
                NodeTag.VALUE_STOP_WORDS.value: 22,

                NodeMachine.VALUE_MACHINE.value: 7,
                NodeIssue.VALUE_MACHINE_DOWN.value: 8,

                NodeIssue.VALUE_DESCRIPTION_PROBLEM.value: 0,
                NodeIssue.VALUE_DESCRIPTION_SOLUTION.value: 1,
                NodeIssue.VALUE_PART_PROCESS.value: 16,
                
                NodeIssue.VALUE_DATE_MAINTENANCE_WORK_ORDER_ISSUE.value: 9,
                NodeIssue.VALUE_DATE_MAINTENANCE_WORK_ORDER_ISSUE.value + "2": 10,
                NodeIssue.VALUE_DATE_MAINTENANCE_WORK_ORDER_CLOSE.value: 11,
                NodeIssue.VALUE_DATE_MAINTENANCE_WORK_ORDER_CLOSE.value + "2": 12
                }

def date_cleanizer(date, time="0000"):
    """
    clean the date from GS data

    :param date: the date
    :param time: the time
    :return: a DateTime
    """

####    DATE ####
    if date is "":
        return None

    date = date.replace(" ","").split("/")

    date[2] = "20" + date[2]

    if len(date[0]) is 1:
        date[0] = "0" + date[0]
    else:
        date[0] = date[0]

    if len(date[1]) is 1:
        date[1] = "0" + date[1]
    else:
        date[1] = date[1]

#### TIME ####
    while len(time) < 4:
        time = "0" + time.replace(" ", "")

    return datetime(year=int(date[2]), month=int(date[0]), day=int(date[1]), hour=int(time[:2]), minute=int(time[2:]))

### Psu

In [ ]:
PassWord = 'PSU!!'
file = os.path.join(project_path , 'Data_CSV', 'Psu_Data.csv')
localization = {NodeHuman.VALUE_TECHNICIAN.value: 18,

                NodeMachine.VALUE_MACHINE.value: 3,
                NodeMachine.VALUE_TYPE.value:6,
                NodeMachine.VALUE_MANUFACTURER.value:5,
                NodeMachine.VALUE_LOCASION.value:4,


                NodeIssue.VALUE_DESCRIPTION_PROBLEM.value: 11,
                NodeIssue.VALUE_DESCRIPTION_SOLUTION.value: 19,
                NodeIssue.VALUE_DESCRIPTION_CAUSE.value:12,
                NodeIssue.VALUE_MACHINE_DOWN.value: 13,
                }

### Hvac

In [ ]:
PassWord = 'HVAC!!'
file = os.path.join(project_path , 'Data_CSV', 'Hvac_Data.csv')
localization = {NodeHuman.VALUE_TECHNICIAN.value: 10,
                NodeHuman.VALUE_OPERATOR.value: 43,

                NodeMachine.VALUE_MACHINE.value: 7,

                NodeIssue.VALUE_DESCRIPTION_PROBLEM.value: 5,
                NodeIssue.VALUE_DESCRIPTION_SOLUTION.value: 247,
                }
date_cleanizer = None

### Auto

In [ ]:
PassWord = 'AUTO!!'
file = os.path.join(project_path , 'Data_CSV', 'Auto_Data.csv')
localization = {NodeHuman.VALUE_TECHNICIAN.value: 4,
                NodeHuman.VALUE_OPERATOR.value: 11,
                NodeHuman.VALUE_CRAFTS.value: 16,
                NodeHuman.VALUE_SKILLS.value: 17,

                NodeMachine.VALUE_MACHINE.value: 8,

                NodeIssue.VALUE_DESCRIPTION_PROBLEM.value: 5,
                NodeIssue.VALUE_DESCRIPTION_SOLUTION.value: 7,
                }

### Mine

In [ ]:
PassWord = 'MINE!!'
file = os.path.join(project_path , 'Data_CSV', 'Mine_Data.csv')

localization = {}

## ACTION

### Connect

In [3]:
database = DatabaseNeo4J("bolt://127.0.0.1:7687", "neo4j", PassWord)

In [ ]:
database = DatabaseNeo4J("bolt://127.0.0.1:7687", "neo4j", "11/Jul/2017!")

## Indexs

In [ ]:
database.createIndexes()

In [ ]:
database.dropIndexes()

## Data

In [ ]:
graph_database_from_csv(database, file, localization, date_cleanizer)

In [ ]:
# token_dataframe = pd.DataFrame.from_csv("GS_tokenized_forSascha.csv", index_col =None)
# graph_database_from_tag_extractor(database = database, token_dataframe = token_dataframe)

In [ ]:
database.deleteData()

# KPI

# KPI using Object

In [ ]:
tec = Technician(name=["_"])
ope = Operator(name=["_"])

i3=Issue(problem=["_"], solution=["_"], machine_down="_",date_maintenance_work_order_issue=["_"], date_maintenance_work_order_close=["_"],part_in_process=["_"] )
i=Issue(date_maintenance_work_order_issue=["_"])
mac = Machine(name=["_","h11", "h1"])

tagI = TagItem(keyword = ["_"])
tagS = TagAction(keyword = ["_"] , it_is="s")
tagP = TagAction(keyword = ["_"] , it_is="p")



In [ ]:
query = cypher_from_kpi(tec,ope)

In [ ]:
dataframe = pandas_from_cypher_kpi(database, query)

In [ ]:
dataframe.shape

In [ ]:
print(query)

In [ ]:
columns = dataframe.keys().tolist()
dataframe["index"] = 1

In [ ]:
dataframe.head()

In [ ]:
df = pd.pivot_table(dataframe, index=dataframe.index, columns=columns, fill_value=0 )

In [ ]:
df

# For UI


In [4]:
MaintenanceWorkOrder.get_dict_database_structure(database)

{'Human': {'name'},
 'Issue': {'date_maintenance_work_order_close',
  'date_maintenance_work_order_issue',
  'description_of_problem',
  'description_of_solution',
  'machine_down',
  'part_in_process'},
 'Item': {'keyword'},
 'Machine': {'name'},
 'Operator': {'name'},
 'Problem': {'keyword'},
 'Solution': {'keyword'},
 'Tag': {'keyword'},
 'Technician': {'name'},
 'Technician and Operator': {'name'}}

In [24]:
import pandas as pd
df = pd.DataFrame({"a":["flow","tank","pump"], 'b':[12,59,3]})

In [25]:
from matplotlib.figure import Figure
fig = Figure(figsize=(5, 4), dpi=100)
axes = fig.add_subplot(111)
axes.plot(x=df["b"], y=df["b"] )

[]

In [27]:
axes.show()

AttributeError: 'AxesSubplot' object has no attribute 'show'

In [21]:
x=df["b"]

In [22]:
x

0    12
1    59
2     3
Name: b, dtype: int64